In [4]:
from __future__ import absolute_import, division, print_function

import sys
import os
import time

sys.path.insert(0,'../../../')
sys.path.insert(0,'../../../py')

import parameters
import utilities
import spectrogram_utilities
import output_utilities
import spectrogram_output_visualiser
import spectrogram_cuts_db_creation

import tensorflow as tf
import numpy as np
import pandas as pd
import h5py


from scipy import signal
from sklearn import preprocessing
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# matplotlib.get_backend()
%matplotlib inline
#

In [2]:
# constants
awd_event = 1
site = parameters.sites[0]

## Database

### Creating TFRecords files

In [3]:
files = utilities.all_files(awd_event, site)
spectrogram_cuts_db_creation.spectrogram_cuts_h5py(awd_event, site,verbose=True)
# spectrogram_cuts_db_creation.spectrogram_cuts_tfrecords(awd_event, site,verbose=True)


Generating whistler and noise cuts database for awdEvent1/marion
0%.........10%.........20%.........30%.........40%.........50%.........60%.........70%.........80%.........90%.........
Runtime: 252.38 seconds


### Reading TFRecord files

In [ ]:
NUM_SHUFFLE = 100
NUM_EPOCHS = 50
BATCH_SIZE = 5
path = os.path.join(parameters.tfrecord_location,'awdEvents'+str(awd_event),'cuts')
files = tf.data.Dataset.list_files(os.path.join(path,'*.tfr'))
print(type(files))
dataset = tf.data.TFRecordDataset(files, num_parallel_reads=8)

dataset = dataset.shuffle(NUM_SHUFFLE)
dataset = dataset.repeat(NUM_EPOCHS)
features = {
    'data': tf.FixedLenFeature([], tf.float32),
    'merit': tf.FixedLenFeature([], tf.int64),
    'label': tf.FixedLenFeature([], tf.int64)
    }
dataset = dataset.map(lambda x: tf.parse_single_example(x, features))
dataset = dataset.batch(BATCH_SIZE)

iterator = dataset.make_one_shot_iterator()
iterator.get_next()

## Convolutional Neural Network

In [6]:
assert features.shape[0]==labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))


In [ ]:
input_layer = tf.reshape(data,[-1, f_cut_length, t_cut_length, 1])

In [8]:
def cnn_model_fn(features, labels, mode):
    '''Model function for CNN.'''
    
    # input layer
    input_layer = tf.reshape(features,[-1, f_cut_length, t_cut_length, 1])
    
    # convolutional layer #1
    conv1 = tf.layers.conv2d(
                inputs=input_layer,
                filters=8,
                kernel_size=[5,5],
                padding='same',
                activation=tf.nn.relu)
    
    # pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    
    # convolutional layer #2 and pooling layer #2
    conv2 = tf.layers.conv2d(
                inputs=pool1,
                filters=16,
                kernel_size=[5,5],
                padding='same',
                activaltion=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    
    # denser layer
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # logit layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    predictions = {
        # generate predictions for (PREDICT and EVAL mode)
        'classes': tf.argmax(input=logits, axis=1),
        # add 'softmax_tensor' to the graph. It is used for PREDICT and by the 'looging_hook'
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # configure the training optimizer (for TRAIN mode)
    if mode==tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    

In [ ]:

data = preprocessing.scale(data)

start = time.time()
end = time.time()
print("\nRuntime: {:.2f} seconds".format(end - start))
